In [1]:
%reload_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import sys
code_dir = os.path.dirname(os.getcwd())
sys.path.append(code_dir)

from glob import glob
import json
import glob
import wave
import math
from itertools import permutations

import numpy as np
from scipy import stats, signal

import essentia.standard as es
import librosa

import tensorflow as tf
#from model.fp.melspec.melspectrogram import get_melspec_layer, Melspec_layer_essentia
from run import load_config
from model.utils.audio_utils import load_audio, background_mix, max_normalize, ir_aug

import IPython.display as ipd
from model.utils.audio_utils import load_audio, get_fns_seg_dict

from model.dataset import Dataset

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


In [5]:
import yaml

def load_config(config_filepath):
    #config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')
    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

cfg = load_config('/home/oaraz/nextcore/afp/neural-audio-fp/config/default.yaml')

dataset = Dataset(cfg)

# train_ds = dataset.get_train_ds(cfg['TRAIN']['REDUCE_ITEMS_P'])
# val_ds = dataset.get_val_ds()

cli: Configuration from /home/oaraz/nextcore/afp/neural-audio-fp/config/default.yaml


In [6]:
dataset.get_train_ds()

Creating the training dataset...
tr_bg_fps:      0


AssertionError: No background noise found.

In [2]:
DISCOTUBE = "/mnt/projects/discotube/"
discotube_2020 = DISCOTUBE + "discotube-2020-09/audio"
discotube_2023 = DISCOTUBE + "discotube-2023-03/audio-new/audio/"

# discotube_2023_mp4_paths = glob(os.path.join(discotube_2023, "**/*.mp4"), recursive=True)
# print(len(discotube_2023_mp4_paths))
# discotube_2020_mp4_paths = glob(os.path.join(discotube_2020, "**/*.mp4"), recursive=True)
# print(len(discotube_2020_mp4_paths))

In [14]:
with open("/home/oaraz/nextcore/afp/discotube-2023.txt", "r") as in_f, open("/home/oaraz/nextcore/afp/discotube-2023.txt.filtered", "w") as out_f:
    for mp4_path in in_f.readlines():
        meta_path = mp4_path[:-4]+"meta"
        with open(meta_path, "r") as meta_f:
            metadata = json.load(meta_f)
        if metadata["duration"] < 420:
            out_f.write(mp4_path)
        break

In [12]:
type(metadata["duration"])

int

In [13]:
metadata["id"]

'F_oWcMGPn3I'

In [10]:
list(metadata.keys())

['id',
 'title',
 'formats',
 'thumbnails',
 'thumbnail',
 'description',
 'uploader',
 'uploader_id',
 'uploader_url',
 'channel_id',
 'channel_url',
 'duration',
 'view_count',
 'average_rating',
 'age_limit',
 'webpage_url',
 'categories',
 'tags',
 'playable_in_embed',
 'live_status',
 'release_timestamp',
 '_format_sort_fields',
 'automatic_captions',
 'subtitles',
 'album',
 'artist',
 'track',
 'release_date',
 'release_year',
 'comment_count',
 'chapters',
 'like_count',
 'channel',
 'channel_follower_count',
 'upload_date',
 'creator',
 'alt_title',
 'availability',
 'original_url',
 'webpage_url_basename',
 'webpage_url_domain',
 'extractor',
 'extractor_key',
 'playlist',
 'playlist_index',
 'display_id',
 'fulltitle',
 'duration_string',
 'is_live',
 'was_live',
 'requested_subtitles',
 '_has_drm',
 'asr',
 'filesize',
 'format_id',
 'format_note',
 'source_preference',
 'fps',
 'audio_channels',
 'height',
 'quality',
 'has_drm',
 'tbr',
 'url',
 'width',
 'language',
 'la

In [20]:
cfg_path = "default"
cfg = load_config(cfg_path)

cli: Configuration from ./config/default.yaml


In [5]:
_prefix = 'train-10k-30s/'
source_root_dir = cfg["DIR"]["SOURCE_ROOT_DIR"]

tr_source_fps = sorted(
            glob.glob(source_root_dir + _prefix + '**/*.wav',
                      recursive=True))
print(len(tr_source_fps))

fns_event_seg_dict = get_fns_seg_dict(tr_source_fps,
                                    "all",
                                    8000,
                                    1.0,
                                    0.5)
print(len(fns_event_seg_dict))

10000
10000


In [7]:
shorter_files = []
for fname,segments in fns_event_seg_dict.items():
    if len(segments)==58:
        shorter_files.append(fname)
        print(fname, len(segments))

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/000/000002.wav 58
../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/000/000005.wav 58
../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/000/000010.wav 58
../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/000/000140.wav 58
../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/000/000141.wav 58
../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/000/000148.wav 58
../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/000/000190.wav 58
../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/000/000193.wav 58
../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/000/000194.wav 58
../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/000/000197.wav 58
../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_mediu

In [9]:
for fname in shorter_files:
    pt_wav = wave.open(fname, 'r')
    n_frames = pt_wav.getnframes()
    print(n_frames/8000)
    pt_wav.close()

29.9625
29.988625
29.9625
29.936375
29.936375
29.9625
29.988625
29.9625
29.988625
29.9625
29.9625
29.9625
29.9625
29.9625
29.988625
29.9625
29.9625
29.9625
29.9625
29.9625
29.988625
29.9625
29.988625
29.988625
29.988625
29.9625
29.9625
29.9625
29.9625
29.988625
29.9625
29.9625
29.988625
29.988625
29.9625
29.988625
29.988625
29.9625
29.9625
29.9625
29.9625
29.9625
29.9625
29.9625
29.9625
29.988625
29.9625
29.9625
29.988625
29.9625
29.9625
29.988625
29.9625
29.9625
29.9625
29.988625
29.988625
29.988625
29.9625
29.9625
29.9625
29.9625
29.988625
29.9625
29.988625
29.988625
29.988625
29.9625
29.9625
29.9625
29.9625
29.988625
29.9625
29.988625
29.9625
29.988625
29.9625
29.9625
29.988625
29.988625
29.988625
29.9625
29.9625
29.988625
29.988625
29.988625
29.9625
29.9625
29.988625
29.988625
29.9625
29.988625
29.9625
29.988625
29.9625
29.9625
29.9625
29.9625
29.9625
29.9625
29.9625
29.988625
29.976
29.988625
29.9625
29.988625
29.9625
29.9625
29.9625
29.988625
29.9625
29.9625
29.988625
29.9625
29.

In [16]:
dataset = Dataset(cfg)

train_ds = dataset.get_train_ds()
print(len(train_ds))
print(len(train_ds.bg_clips))
print(len(train_ds.ir_clips))

166
2142
345


In [13]:
batch = train_ds.__getitem__(0)
print(batch[0].shape)
print(batch[1].shape)
print(batch[2].shape)
print(batch[3].shape)

(60, 8000)
(60, 8000)
(60, 256, 32, 1)
(60, 256, 32, 1)


In [20]:
fname = train_ds.event_fnames[0]
print(fname)
print(train_ds.fns_event_seg_dict[fname][:3])
train_ds.shuffle_events()
print(train_ds.event_fnames[0])
print(train_ds.fns_event_seg_dict[fname][:3])


../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/011/011781.wav
[[8, -4000, 4000.0], [28, -4000, 4000.0], [57, -4000, 3700.0]]
../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/116/116101.wav
[[8, -4000, 4000.0], [28, -4000, 4000.0], [57, -4000, 3700.0]]


In [9]:
fname = '../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/036/036799.wav'
print(train_ds.fns_event_seg_dict[fname][:5])
train_ds.shuffle_segments()
print(train_ds.fns_event_seg_dict[fname][:5])


[[4, -4000, 4000.0], [18, -4000, 4000.0], [42, -4000, 4000.0], [24, -4000, 4000.0], [57, -4000, 4000.0]]
[[52, -4000, 4000.0], [56, -4000, 4000.0], [5, -4000, 4000.0], [24, -4000, 4000.0], [29, -4000, 4000.0]]


In [21]:
train_ds.shuffle_events()

In [22]:
fname = train_ds.bg_fnames[1]
print(train_ds.fns_bg_seg_dict[fname][:5])
train_ds.shuffle_segments()
print(train_ds.fns_bg_seg_dict[fname][:5])


[[8, -8000, 8000.0], [6, -8000, 8000.0], [4, -8000, 8000.0], [9, -8000, 0.0], [7, -8000, 8000.0]]
[[7, -8000, 8000.0], [2, -8000, 8000.0], [5, -8000, 8000.0], [8, -8000, 8000.0], [3, -8000, 8000.0]]


In [24]:
for bg_clip in train_ds.bg_clips.values():
    print(len(bg_clip)/8000)

10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0


In [ ]:
train_ds = dataset.get_train_ds()
#val_ds = dataset.get_val_ds() # max 500

In [10]:
dataset.tr_source_fps

In [12]:
fns_event_seg_dict = get_fns_seg_dict(dataset.tr_source_fps,
                                    "all",
                                    8000,
                                    1.0,
                                    0.5)
print(len(fns_event_seg_dict))

10000


In [17]:
for fname,segments in fns_event_seg_dict.items():
    if len(segments)<58:
        print(fname, len(segments))

../neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/098/098566.wav 11


In [ ]:
for fname, seg_idx, offset_min, offset_max in train_ds.fns_event_seg_list:

    if os.path.basename(fname) == '114558.wav':
    #if offset_max==0:# and offset_max==0:
        print(fname)
        print(seg_idx)
        print(offset_min)
        print(offset_max)
        print()

In [ ]:
len(train_ds.fns_event_seg_list)

In [ ]:
np.max([-4000, -train_ds.offset_margin_frame])

In [ ]:
np.min([0.0, train_ds.offset_margin_frame])

In [ ]:
for fname, seg_idx, offset_min, offset_max in train_ds.fns_event_seg_list:

    if os.path.basename(fname) == '114558.wav':
    #if offset_max==0:# and offset_max==0:
        print(fname)
        print(seg_idx)
        print(offset_min)
        print(offset_max)
        print()

In [138]:
dataset = Dataset(cfg)

# train_ds = dataset.get_train_ds(reduce_items_p=0.1)
train_ds = dataset.get_train_ds()

print(f"Number of tracks: {len(train_ds.track_fnames)}")
print(f"Number of batches: {len(train_ds)}")
print(len(train_ds.track_fnames)*train_ds.segments_per_track/train_ds.n_anchor)
print(f"Tracks per batch: {train_ds.n_anchor}")
print(f"Segments per track: {train_ds.segments_per_track}")

print(f"Number of iterations required to see all tracks once: {len(train_ds.track_fnames)//train_ds.n_anchor}")

Increase training data or decrease segments_per_track to avoid this warning.
Number of tracks: 9960
Number of batches: 9628
9628.0
Tracks per batch: 60
Segments per track: 58
Number of iterations required to see all tracks once: 166


In [123]:
for i,item in enumerate(train_ds):
    pass

In [131]:
len(train_ds.track_fnames)*58/len(train_ds.bg_fnames)

269.69187675070026

In [126]:
for fname in train_ds.bg_fnames:

    if not set([k for k in train_ds.bg_segments[fname].keys()])==set([seg_idx for seg_idx,_,_ in train_ds.fns_bg_seg_dict[fname]]):
        print(fname)

../neural-audio-fp-dataset/aug/bg/tr/pub_noise_8k/noise1_0.wav
../neural-audio-fp-dataset/aug/bg/tr/pub_noise_8k/noise1_3.wav
../neural-audio-fp-dataset/aug/bg/tr/pub_noise_8k/noise3_9.wav
../neural-audio-fp-dataset/aug/bg/tr/pub_noise_8k/noise2_5.wav
../neural-audio-fp-dataset/aug/bg/tr/pub_noise_8k/noise3_6.wav
../neural-audio-fp-dataset/aug/bg/tr/pub_noise_8k/noise3_5.wav
../neural-audio-fp-dataset/aug/bg/tr/pub_noise_8k/noise3_2.wav
../neural-audio-fp-dataset/aug/bg/tr/pub_noise_8k/noise1_5.wav
../neural-audio-fp-dataset/aug/bg/tr/pub_noise_8k/noise1_8.wav
../neural-audio-fp-dataset/aug/bg/tr/pub_noise_8k/noise3_3.wav
../neural-audio-fp-dataset/aug/bg/tr/pub_noise_8k/noise2_0.wav
../neural-audio-fp-dataset/aug/bg/tr/pub_noise_8k/noise1_4.wav
../neural-audio-fp-dataset/aug/bg/tr/pub_noise_8k/noise2_4.wav
../neural-audio-fp-dataset/aug/bg/tr/pub_noise_8k/noise3_7.wav
../neural-audio-fp-dataset/aug/bg/tr/pub_noise_8k/noise3_0.wav
../neural-audio-fp-dataset/aug/bg/tr/pub_noise_8k/noise

In [129]:
len(train_ds.fns_bg_seg_dict['../neural-audio-fp-dataset/aug/bg/tr/pub_noise_8k/noise1_3.wav'])

300

In [117]:
print(set([val for val in train_ds.bg_seen.values()]))
print(set([val for val in train_ds.bg_seen.keys()])==set(train_ds.bg_fnames))

{269, 270}
True


In [119]:
len(train_ds.bg_fnames)

2142

In [110]:
print(set([v for v in train_ds.seen_tracks.values()]))
print(set([v for v in train_ds.seen_tracks.keys()])==set(train_ds.track_fnames))
print(set([len(train_ds.seen_track_segments[fname]) for fname in train_ds.track_fnames]))

{58}
True
{58}


In [107]:
train_ds.seen_track_segments[fname]

Counter({11: 1,
         8: 1,
         16: 1,
         18: 1,
         13: 1,
         7: 1,
         1: 1,
         20: 1,
         15: 1,
         6: 1,
         21: 1,
         14: 1,
         2: 1,
         12: 1,
         10: 1,
         19: 1,
         4: 1,
         9: 1,
         0: 1,
         5: 1,
         17: 1,
         3: 1})

In [26]:
len(train_ds.track_seg_dict[train_ds.track_fnames[0]])

58

In [43]:
for i,item in enumerate(train_ds):
    print(i%train_ds.segments_per_track)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
0
1
2
3
4
5
6
7


In [46]:
train_ds.track_seg_dict[train_ds.track_fnames[0]]

[[1, -4000, 4000.0],
 [35, -4000, 4000.0],
 [16, -4000, 4000.0],
 [49, -4000, 4000.0],
 [0, 0, 4000.0],
 [30, -4000, 4000.0],
 [20, -4000, 4000.0],
 [48, -4000, 4000.0],
 [9, -4000, 4000.0],
 [36, -4000, 4000.0],
 [57, -4000, 4000.0],
 [53, -4000, 4000.0],
 [14, -4000, 4000.0],
 [12, -4000, 4000.0],
 [44, -4000, 4000.0],
 [4, -4000, 4000.0],
 [43, -4000, 4000.0],
 [3, -4000, 4000.0],
 [38, -4000, 4000.0],
 [55, -4000, 4000.0],
 [21, -4000, 4000.0],
 [29, -4000, 4000.0],
 [2, -4000, 4000.0],
 [32, -4000, 4000.0],
 [50, -4000, 4000.0],
 [17, -4000, 4000.0],
 [39, -4000, 4000.0],
 [42, -4000, 4000.0],
 [46, -4000, 4000.0],
 [7, -4000, 4000.0],
 [23, -4000, 4000.0],
 [13, -4000, 4000.0],
 [22, -4000, 4000.0],
 [45, -4000, 4000.0],
 [34, -4000, 4000.0],
 [25, -4000, 4000.0],
 [27, -4000, 4000.0],
 [33, -4000, 4000.0],
 [10, -4000, 4000.0],
 [51, -4000, 4000.0],
 [11, -4000, 4000.0],
 [8, -4000, 4000.0],
 [15, -4000, 4000.0],
 [37, -4000, 4000.0],
 [24, -4000, 4000.0],
 [52, -4000, 4000.0],


In [ ]:
train_ds.seen_tracks==set(train_ds.track_fnames)

## FMA Tests

In [28]:
import pandas as pd

In [46]:
df_full = pd.read_csv("/home/oaraz/nextcore/fingerprinting/neural-audio-fp/extras_dataset/fma_info/tracks.csv", low_memory=False)

In [47]:
df_full.head()

,Unnamed: 0,album,album.1,album.2,album.3,album.4,album.5,album.6,album.7,album.8,...,track.10,track.11,track.12,track.13,track.14,track.15,track.16,track.17,track.18,track.19
0,NaN,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,...,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
1,track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
3,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
4,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World


In [54]:
df_full.shape

(106576, 53)

In [51]:
df_full.columns

Index(['Unnamed: 0', 'album', 'album.1', 'album.2', 'album.3', 'album.4',
       'album.5', 'album.6', 'album.7', 'album.8', 'album.9', 'album.10',
       'album.11', 'album.12', 'artist', 'artist.1', 'artist.2', 'artist.3',
       'artist.4', 'artist.5', 'artist.6', 'artist.7', 'artist.8', 'artist.9',
       'artist.10', 'artist.11', 'artist.12', 'artist.13', 'artist.14',
       'artist.15', 'artist.16', 'set', 'set.1', 'track', 'track.1', 'track.2',
       'track.3', 'track.4', 'track.5', 'track.6', 'track.7', 'track.8',
       'track.9', 'track.10', 'track.11', 'track.12', 'track.13', 'track.14',
       'track.15', 'track.16', 'track.17', 'track.18', 'track.19'],
      dtype='object')

In [48]:
df_full.iloc[0]

Unnamed: 0                  NaN
album                  comments
album.1            date_created
album.2           date_released
album.3                engineer
album.4               favorites
album.5                      id
album.6             information
album.7                 listens
album.8                producer
album.9                    tags
album.10                  title
album.11                 tracks
album.12                   type
artist        active_year_begin
artist.1        active_year_end
artist.2      associated_labels
artist.3                    bio
artist.4               comments
artist.5           date_created
artist.6              favorites
artist.7                     id
artist.8               latitude
artist.9               location
artist.10             longitude
artist.11               members
artist.12                  name
artist.13      related_projects
artist.14                  tags
artist.15               website
artist.16        wikipedia_page
set     

In [58]:
df_full['set'].unique()

array(['split', nan, 'training', 'validation', 'test'], dtype=object)

In [59]:
print(df_full[df_full['set']=="training"].shape)
print(df_full[df_full['set']=="validation"].shape)
print(df_full[df_full['set']=="test"].shape)

(84353, 53)
(10958, 53)
(11263, 53)


In [60]:
df_full['set.1'].unique()

array(['subset', nan, 'small', 'medium', 'large'], dtype=object)

In [61]:
print(df_full[df_full['set.1']=="small"].shape)
print(df_full[df_full['set.1']=="medium"].shape)
print(df_full[df_full['set.1']=="large"].shape)

(8000, 53)
(17000, 53)
(81574, 53)


In [75]:
total_unique = 81574 + 17000 + 8000
print(total_unique)

106574


all 8000 of small and 2000 from medium used for training
500 val 500 query

total used 11000
total different 106574

In [76]:
total_used = 10000 + 500 + 500
print(total_used)

11000


In [77]:
total_unique - total_used

95574

In [78]:
our_test_set_size = 93458

In [70]:
set(df_full[df_full['set.1']=="small"].index.to_numpy()).intersection(set(df_full[df_full['set.1']=="medium"].index.to_numpy()))

set()

With known header row

In [34]:
df = pd.read_csv("/home/oaraz/nextcore/fingerprinting/neural-audio-fp/extras_dataset/fma_info/tracks.csv", low_memory=False, header=2)

In [35]:
df.head()

,track_id,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52
0,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
1,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
2,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
3,10,0,2008-11-26 01:45:08,2008-02-06 00:00:00,NaN,4,6,NaN,47632,NaN,...,NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
4,20,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,NaN,...,NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,NaN,3,NaN,[],Spiritual Level


In [36]:
df.columns

Index(['track_id', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29',
       'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37',
       'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41',
       'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45',
       'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49',
       'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52'],
      dtype='object')

In [7]:
df.shape

(106576, 53)

In [38]:
df['track_id']

0              2
1              3
2              5
3             10
4             20
           ...  
106569    155316
106570    155317
106571    155318
106572    155319
106573    155320
Name: track_id, Length: 106574, dtype: int64

In [26]:
import glob
import os
from itertools import combinations

In [23]:
music_dir = "/home/oaraz/nextcore/fingerprinting/datasets/neural-audio-fp-dataset/music"
train_dir = os.path.join(music_dir, "train-10k-30s")
val_dir = os.path.join(music_dir, "val-query-db-500-30s")
test_dummy_dir = os.path.join(music_dir, "test-dummy-db-100k-full")
test_query_db_dir = os.path.join(music_dir, "test-query-db-500-30s/db")

In [24]:
train_fps = sorted(glob.glob(os.path.join(train_dir, "**/*.wav"), recursive=True))
val_fps = sorted(glob.glob(os.path.join(val_dir, "**/*.wav"), recursive=True))
test_dummy_fps = sorted(glob.glob(os.path.join(test_dummy_dir, "**/*.wav"), recursive=True))
test_query_db_fps = sorted(glob.glob(os.path.join(test_query_db_dir, "**/*.wav"), recursive=True))

In [25]:
train_fnames = set([os.path.basename(fp) for fp in train_fps])
print(len(train_fnames))
val_fnames = set([os.path.basename(fp) for fp in val_fps])
print(len(val_fnames))
test_dummy_fnames = set([os.path.basename(fp) for fp in test_dummy_fps])
print(len(test_dummy_fnames))
test_query_db_fnames = set([os.path.basename(fp) for fp in test_query_db_fps])
print(len(test_query_db_fnames))

10000
500
93458
500


In [27]:
for a,b in combinations([train_fnames, val_fnames, test_dummy_fnames, test_query_db_fnames], 2):
    print(len(a.intersection(b)))

0
0
0
0
0
0


In [45]:
for fnames in [train_fnames, val_fnames, test_dummy_fnames, test_query_db_fnames]:
    for fname in fnames:
        fname = os.path.splitext(fname)[0]
        if df[df['track_id']==int(fname)].size==0:
            print("wtf")